In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import numpy as np
import matplotlib
import pickle
from sklearn.manifold import TSNE 
from sklearn.decomposition import PCA

sns.set(font_scale=1, context='notebook')
matplotlib.rcParams['figure.figsize'] = 10,10
plt.rcParams['axes.facecolor'] = '#f5f5f5'

saved_filename = 'test'

# Define Functions

In [ ]:
def visualize_gates(gates, images, num_test_digits=10, shuffle=True):
    '''
    Visualize num_test_digits examples of gate activations
    '''
    # Indiviual cell is height=1, width=0.4
    width = 1 * gates.shape[2] # Cell width * num of modules
    height = 0.4 * gates.shape[1] # Cell height * num of layers
    figsize = (width, height) 

    plot_num = num_test_digits * 100 + 11
    for i in range(num_test_digits):
        # To shuffle
        if shuffle:
            k = (i + np.random.randint(low=0, high=gates.shape[0]-1)) % gates.shape[0]
        else:
            k = i

        plt.figure(figsize=(5, 2))
        plt.title('Number is: '+str(labels[k]))
        sns.heatmap(gates[k,:,:])
        plt.figure(figsize=(2, 2))
        plt.imshow(images[k])


def compute_cosine_sim(gates, parameter_dict, num_test_digits, trial = 2):
    '''
    Compute cosine similarity
    '''
    gate_vectors = np.reshape(gates, (-1, parameter_dict['L']*parameter_dict['M']))
    #print(gate_vectors.shape)
    gate_vectors = np.transpose(gate_vectors)

    output = np.zeros(num_test_digits)
    for i in range(num_test_digits):
        a = gate_vectors[:, trial]
        b = gate_vectors[:, i]
        an = a/np.linalg.norm(a)
        bn = b/np.linalg.norm(b)
        output[i] = np.dot(an,bn)

    print('output', output)
    elems = np.argsort(output)
    print('Most similar elems to trial', elems[::-1])
    
    
def visualize_model(gates_reshaped, labels, model):
    '''
    Visualize the data X, with label y with the model (t-SNE or PCA)
    '''
    print(gates_reshaped.shape) # sanity check
    independent_labels = list(set(labels))
    print(independent_labels) # sanity check

    x = model.fit_transform(gates_reshaped)
    y = labels

    # Add more colors if doing something other than MNIST
    colors = ['b','g','#00FFFF','c','m','y','k','#00A4BA','r','#F4C2C2']

    fig = plt.figure()
    ax1 = fig.add_subplot(111)

    for i in (independent_labels):
        mask_i = y == i
        x_i = x[mask_i]
        ax1.scatter(x_i[:,0], x_i[:,1], s=10, c=colors[i], label='Label: '+str(i))
    plt.legend(loc='upper left');
    plt.show()


def visualize_comparison(gates_reshaped, labels, model, digits=[7,8]):
    '''
    Same as visualize model but only plots certain digits 
    '''
    mask = labels == digits[0]
    for digit in digits:
        mask_digit = labels == digit
        mask = mask | mask_digit

    gates_reshaped_masked = gates_reshaped[mask, :]
    labels_masked = labels[mask]
    
    x = model.fit_transform(gates_reshaped_masked)
    y = labels_masked

    # Add more colors if doing something other than MNIST
    colors = ['b','g','#00FFFF','c','m','y','k','#00A4BA','r','#F4C2C2']

    fig = plt.figure()
    ax1 = fig.add_subplot(111)

    for i in (digits):
        mask_i = y == i
        x_i = x[mask_i]
        ax1.scatter(x_i[:,0], x_i[:,1], s=10, c=colors[i], label='Label: '+str(i))
    plt.legend(loc='upper left');
    plt.show()

<br><br><br>
# ------------------------------------------------------------------------------------
#                               Visualize gates and images
# ------------------------------------------------------------------------------------
<br>

# Load data
<br>

In [ ]:
# Load gate vector data
with open('output/' + saved_filename + '.pkl', 'rb') as f:
        input = pickle.load(f)

# Get components from data
gates = np.array(input[0])
labels = np.array(input[1])
images = np.array(input[2])
parameter_dict = input[3]

print(parameter_dict)
print(labels.shape)
print(gates.shape)

In [ ]:
visualize_gates(gates, images, num_test_digits=5)

In [ ]:
# Close all figures (so they don't stay open in memory)
plt.close('all')

In [ ]:
compute_cosine_sim(gates, parameter_dict, num_test_digits=5, trial=2)

# Visualizations of the gates (dim reduction)


In [ ]:
# Preprocess to make gates the correct size
gates_reshaped = np.reshape(gates, [gates.shape[0], -1])

### Plot PCA

In [ ]:
# Visualize PCA
visualize_model(gates_reshaped, labels, PCA())

### Plot t-SNE

In [ ]:
# Visualize t-SNE
visualize_model(gates_reshaped, labels, TSNE(verbose=100))

### Plot digit comparisons (PCA)

In [ ]:
# Visualize the comparison between a and b (with PCA)
visualize_comparison(gates_reshaped, labels, PCA(), digits=[0,3,8])

### Plot digit comparisons (t-SNE)


In [ ]:
# Visualize the comparison between a and b (with t-SNE)
visualize_comparison(gates_reshaped, labels, TSNE(), digits=[0,3,8])

<br><br><br>
# ------------------------------------------------------------------------------------
#                      For final feature layer (pre-softmax layer)
# ------------------------------------------------------------------------------------
<br>

In [ ]:
# Load gate vector data
with open('output/' + saved_filename + '__feature_layer_outputs' + '.pkl', 'rb') as f:
        input = pickle.load(f)

# Get components from data
features = np.array(input[0])
labels = np.array(input[1])
images = np.array(input[2])
parameter_dict = input[3]

print(parameter_dict)
print(labels.shape)
print(features.shape)

## Visualize predictions and images


In [ ]:
visualize_gates(features, images, num_test_digits=5)

In [ ]:
# Close all figures (so they don't stay open in memory)
plt.close('all')

In [ ]:
features_reshaped = np.reshape(features, [features.shape[0], -1])
print(features.shape)
print(features_reshaped.shape)
print(np.argmax(features, axis=0))

### Plot PCA

In [ ]:
visualize_model(features_reshaped, labels, PCA())

### Plot t-SNE

In [ ]:
visualize_model(features_reshaped, labels, TSNE(verbose=100))

### Plot digit comparisons (PCA)

In [ ]:
# Visualize the comparison between a and b (with PCA)
visualize_comparison(features_reshaped, labels, PCA(), digits=[0,3,8])

### Plot digit comparisons (t-SNE)

In [ ]:
# Visualize the comparison between a and b (with t-SNE)
visualize_comparison(features_reshaped, labels, TSNE(), digits=[0,3,8])

<br><br><br>
# ------------------------------------------------------------------------------------
#                               For Softmax outputs of digits
# ------------------------------------------------------------------------------------
<br>

## Visualize Softmax Predictions

In [ ]:
# Load gate vector data
with open('output/' + saved_filename + '__softmax_outputs' + '.pkl', 'rb') as f:
        input = pickle.load(f)

# Get components from data
preds = np.array(input[0])
labels = np.array(input[1])
images = np.array(input[2])
parameter_dict = input[3]

print(parameter_dict)
print(labels.shape)
print(preds.shape)

## Visualize predictions and images

In [ ]:
visualize_gates(preds, images, num_test_digits=5)

In [ ]:
# Close all figures (so they don't stay open in memory)
plt.close('all')

In [ ]:
preds_reshaped = np.reshape(preds, [preds.shape[0], -1])
print(preds.shape)
print(preds_reshaped.shape)
print(np.argmax(preds, axis=0))

### Plot PCA

In [ ]:
visualize_model(preds_reshaped, labels, PCA())

### Plot t-SNE

In [ ]:
visualize_model(preds_reshaped, labels, TSNE(verbose=100))

### Plot digit comparisons (PCA)

In [ ]:
# Visualize the comparison between a and b (with PCA)
visualize_comparison(preds_reshaped, labels, PCA(), digits=[0,3,8])

### Plot digit comparisons (t-SNE)

In [ ]:
# Visualize the comparison between a and b (with t-SNE)
visualize_comparison(preds_reshaped, labels, TSNE(), digits=[0,3,8])